In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import spacy 
nlp = spacy.load("en_core_web_sm")
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.utils import to_categorical
from keras import layers
import numpy as np

2023-05-13 19:52:09.600911: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-13 19:52:13.380084: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-13 19:52:13.380162: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-13 19:52:18.002555: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

User-defined Functions

In [2]:
def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not (token.is_stop) ]
    return ' '.join(tokens)

def find_max_len_string(texts):
    max_len = len(texts.loc[0])
    max_string = ''
    i = 0
    while i < len(df1):
        s = texts.loc[i]
        # print(s)
        if len(s) < max_len:
            max_len = len(s)
            max_string = s
        i += 1
    return  max_string

def findMinMax(arrays):
    max = None
    max_len = 0
    min_len = len(arrays[0])
    min = arrays[0]
    for array in arrays:
        size = len(array)
        if size > max_len:
            max = array
        if size < min_len:
            min = array
    return max, min

Read Data

In [3]:
df = pd.read_csv('../Assignments/archive/training.1600000.processed.noemoticon.csv', encoding = "cp1252")
first_row = df.columns
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
first_row_df = pd.DataFrame(data=None, columns = df.columns)
first_row_df.loc[0] = first_row
df = pd.concat([first_row_df,df])
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Choosing a user

In [4]:
user_frequency = df.user.value_counts()
preferred_users = user_frequency[user_frequency > 100]
preferred_users = preferred_users[user_frequency < 150]
print(preferred_users.index.values)

['vacant_heart' 'TizBanana' 'michxxblc' 'whitsundays' 'MandyPandy32'
 'brianwelburn' 'roxiijonas' 'SarahSaner' 'original_one' 'kasey79'
 'jesssicababesss' 'x33ieroNINJA' 'tallivansunder' 'ShesElectric_'
 'JeremyCShipp' 'dontforgetchaos' 'semipenguin' 'BBRRIITTTTYY' 'caldjr'
 'joshtastic1' 'baldeggie' 'DsBabyGirl' 'wyndwitch' 'Anime81'
 'stevegarufi' 'Mirna023' 'heykim' 'keren4562' 'WalkingHorse'
 'chiniehdiaz' 'NKANGEL74' 'radha_' 'Wolfgang_' 'AceMas21' 'ComedyQueen'
 'sharonhayes' 'Kikirowr' 'NKArmyTNgirl' 'MsStaceyK' 'jonas_twilight3'
 'WTFJAY' 'JennaMadison' 'jennypoynter' 'krist0ph3r' 'luckygnahhh'
 'hypnophil' 'adlyman' 'funkyfairy24' 'BeckyKingston' 'kat_n'
 'happylovesChuck' 'MCRmuffin' 'chantellmarie' 'gemeg' 'hummingbird604'
 'longestpoem' 'CaraNinaMcfly' 'fadedmoon' 'annasaccone' 'Gen22'
 'BrandyWandLover' 'figPYBFO' 'tweeteradder7' 'tweeteradder2' 'mp3mad'
 'primatage' 'amysav83' 'aussiemcflyfan' 'Samm_xo' 'lopezwilfred'
 'Hollywood_Trey' 'ladybug8320' 'TidyCat' 'QueenBxoxo'

Let's choose user DarkPiano for this experiment

In [5]:
user = 'vacant_heart'
df1 = df[df.loc[:,"user"] == user]
df1 = df1.reset_index()
df1.text

0      @Eyglo how do u open these blip fm streaming f...
1                 Damn! The Firefox poker aint loading! 
2                Damn! The facebook poker aint loading! 
3      @Richard_Gable good morning mate.Dark and clou...
4      @Dr_DinaSadik forensic mein kya hai! bas rat t...
                             ...                        
142    @nessie_111 f1 today?Oh well the world cup's r...
143    @daaku isnt tweetdeck a bit too huge for an in...
144    btw hav found the perfect twitter client for m...
145                                 @drsimonc  well said
146    @jenna_rater quite agree. Was watching an afri...
Name: text, Length: 147, dtype: object

Preprocessing the tweets

In [16]:
df1.text = df1.text.apply(preprocess)

In [20]:
print(df1.text[32])

@Bass _ happen EU election btw ? mean u vote ? sorry , know EU election .


Create training data

In [7]:
seq_length = 30

# Create a sequence of tokens
sequences = []
for i in range(len(df1)):
    tweet = df1.text.loc[i]

    for i in range(seq_length, len(tweet)):
        seq = tweet[i-seq_length:i+1]
        sequences.append(seq)

# Convert sequences into numerical data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sequences = tokenizer.texts_to_sequences(sequences)

sequences = pad_sequences(sequences, maxlen=seq_length+1, padding='pre', truncating='pre')
sequences = np.array(sequences)

# Take the last integer of the arrays as y and rest as X
X = sequences[:,:-1]
y = sequences[:,-1]
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
y = to_categorical(y, num_classes=vocab_size)

2303


In [8]:
print(X.shape, y.shape)

(3450, 30) (3450, 2303)


Define model

In [13]:
model = Sequential([
    layers.Embedding(vocab_size, 180, input_length=seq_length),
    layers.LSTM(250, return_sequences=True),
    layers.LSTM(250),
    layers.Dense(300, activation='relu'),
    layers.Dense(vocab_size, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 30, 180)           414540    
                                                                 
 lstm_6 (LSTM)               (None, 30, 250)           431000    
                                                                 
 lstm_7 (LSTM)               (None, 250)               501000    
                                                                 
 dense_6 (Dense)             (None, 300)               75300     
                                                                 
 dense_7 (Dense)             (None, 2303)              693203    
                                                                 
Total params: 2,115,043
Trainable params: 2,115,043
Non-trainable params: 0
_________________________________________________________________


Train the model

In [14]:
model.fit(X, y , batch_size=128, epochs=100)

Epoch 1/100
27/27 [==============================] - 14s 375ms/step - loss: 7.4338 - accuracy: 0.0122
Epoch 2/100
27/27 [==============================] - 10s 383ms/step - loss: 6.8888 - accuracy: 0.0145
Epoch 3/100
27/27 [==============================] - 11s 389ms/step - loss: 6.8004 - accuracy: 0.0136
Epoch 4/100
27/27 [==============================] - 10s 376ms/step - loss: 6.7732 - accuracy: 0.0116
Epoch 5/100
27/27 [==============================] - 10s 374ms/step - loss: 6.7516 - accuracy: 0.0139
Epoch 6/100
27/27 [==============================] - 10s 371ms/step - loss: 6.6752 - accuracy: 0.0130
Epoch 7/100
27/27 [==============================] - 10s 382ms/step - loss: 6.6028 - accuracy: 0.0136
Epoch 8/100
27/27 [==============================] - 10s 387ms/step - loss: 6.5437 - accuracy: 0.0171
Epoch 9/100
27/27 [==============================] - 11s 389ms/step - loss: 6.4513 - accuracy: 0.0194
Epoch 10/100
27/27 [==============================] - 10s 373ms/step - loss: 6.255

Accuracy is used as a metric, it seems to be at 79%

Generate New Message

In [51]:
seed_text = "hello"
next_words = 5
for _ in range(next_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre', truncating='pre')
    y_pred = model.predict(encoded, verbose=0)
    predicted_word = ""
    for word, index in tokenizer.word_index.items():
        if index == np.argmax(y_pred):
            predicted_word = word
            break
    seed_text += " " + predicted_word

print(seed_text)
# print(df1.text[0])


hello sorr online tim leg ever
